In [7]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd
from LazyProphet import LazyProphet as lp

train_df = pd.read_csv(r'm4-weekly-train.csv')
test_df = pd.read_csv(r'm4-weekly-test.csv')
train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)

In [6]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [8]:
smapes = []
naive_smape = []
j = tqdm(range(len(train_df)))
for row in j:
    y = train_df.iloc[row, :].dropna()
    y_test = test_df.iloc[row, :].dropna()
    j.set_description(f'{np.mean(smapes)}, {np.mean(naive_smape)}')
    lp_model = lp.LazyProphet(scale=True,
                           seasonal_period=52,
                           n_basis=10,
                           fourier_order=10,
                           ar=list(range(1, 53)),
                           decay=.99,
                           linear_trend=None,
                           decay_average=False)
    fitted = lp_model.fit(y)
    predictions = lp_model.predict(len(y_test)).reshape(-1)
    smapes.append(smape(y_test.values,      pd.Series(predictions).clip(lower=0)))
    naive_smape.append(smape(y_test.values, np.tile(y.iloc[-1], len(y_test))))
print(np.mean(smapes))
print(np.mean(naive_smape))

7.7484259740563886, 9.13930537684544: 100%|██████████| 359/359 [00:42<00:00,  8.49it/s]  

7.7943668416610095
9.161286913982


In [9]:
train_df = pd.read_csv(r'm4-hourly-train.csv')
test_df = pd.read_csv(r'm4-hourly-test.csv')
train_df.index = train_df['V1']
train_df = train_df.drop('V1', axis = 1)
test_df.index = test_df['V1']
test_df = test_df.drop('V1', axis = 1)
smapes = []
naive_smape = []
j = tqdm(range(len(train_df)))
for row in j:
    y = train_df.iloc[row, :].dropna()
    y_test = test_df.iloc[row, :].dropna()
    j.set_description(f'{np.mean(smapes)}, {np.mean(naive_smape)}')
    lp_model = lp.LazyProphet(seasonal_period=[24,168],
                            n_basis=10,
                            fourier_order=10,
                            ar=list(range(1, 25)),
                            decay=.99)
    fitted = lp_model.fit(y)
    predictions = lp_model.predict(len(y_test)).reshape(-1)
    smapes.append(smape(y_test.values, pd.Series(predictions).clip(lower=0)))
    naive_smape.append(smape(y_test.values, np.tile(y.iloc[-1], len(y_test))))
print(np.mean(smapes))
print(np.mean(naive_smape))

11.630263556364747, 42.86116392273107: 100%|██████████| 414/414 [00:47<00:00,  8.73it/s] 

11.633057576291533
43.002986836424824
